<a href="https://colab.research.google.com/github/Michellynss/MJD_Etica/blob/main/Raspador_BBC_Arg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import requests
import bs4
import time
import pandas as pd

In [42]:
import spacy.cli
spacy.cli.download("pt_core_news_lg")

import spacy
nlp = spacy.load("pt_core_news_lg")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [27]:
def grava_arquivo(nome, titulo, texto):
  conteudo = titulo + "\n"

  for paragrafo in texto:
    conteudo = conteudo + paragrafo + "\n"

  with open(nome,'w') as f:
          f.write(conteudo)

In [ ]:
result = requests.get("https://www.bbc.com/portuguese/topics/c2dwqd8nk09t")
soup = bs4.BeautifulSoup(result.text)
links = soup.find_all('a', { 'class': 'focusIndicatorDisplayBlock bbc-uk8dsi e1d658bg0'})
numero_de_artigos = 0
lista_dos_nomes_arquivos_criados = []
for link in links:
    time.sleep(2)
    print(link['href'])
    result2 = requests.get(link['href'])
    soup2 = bs4.BeautifulSoup(result2.text)
    if not( (soup2.find('h1',attrs={"class":"bbc-14gqcmb e1p3vdyi0"})) is None ):
          titulo = soup2.find('h1',attrs={"class":"bbc-14gqcmb e1p3vdyi0"}).text
          texto = []
          for linha in soup2.find_all('div',attrs={"class":"bbc-19j92fr ebmt73l0"}):
            texto.append(linha.text)
          numero_de_artigos = numero_de_artigos + 1
          nome_do_arquivo = 'arq'+str(numero_de_artigos)+".txt"
          lista_dos_nomes_arquivos_criados.append(nome_do_arquivo)
          grava_arquivo(nome_do_arquivo, titulo, texto)
          print("................")

In [38]:
def le_arq(nome):
  arq = open(nome, "r")
  dados = arq.read()
  arq.close()
  return (dados)

In [37]:
df = pd.DataFrame(columns=['artigo','termo','categoria'])

In [ ]:
for nome_arquivo in lista_dos_nomes_arquivos_criados:
  texto = le_arq(nome_arquivo)
  print(nome_arquivo)
  doc = nlp(texto)
  for entity in doc.ents:
      print(entity, entity.label_)
      df.loc[len(df.index)] = [nome_arquivo, entity, entity.label_]

In [44]:
df.head()

,artigo,termo,categoria
0,arq1.txt,(Milei),LOC
1,arq1.txt,(Trump),LOC
2,arq1.txt,(Bolsonaro),PER
3,arq1.txt,"(Javier, Milei)",PER
4,arq1.txt,(Argentina),LOC


In [49]:
df.to_csv('arg.csv')

In [50]:
print(df['termo'].value_counts())

(Milei)                   1
(Heine)                   1
(Brasil)                  1
(BBC, News, Brasil)       1
(Argentina)               1
                         ..
(Argentina)               1
(Argentina)               1
(Silvio, Campos, Neto)    1
(Federico, Servideo)      1
(Etiópia)                 1
Name: termo, Length: 3005, dtype: int64


In [51]:
df['artigo'].unique()

array(['arq1.txt', 'arq2.txt', 'arq3.txt', 'arq4.txt', 'arq5.txt',
       'arq6.txt', 'arq7.txt', 'arq8.txt', 'arq9.txt', 'arq10.txt',
       'arq11.txt', 'arq12.txt', 'arq13.txt', 'arq14.txt', 'arq15.txt',
       'arq16.txt', 'arq17.txt', 'arq18.txt', 'arq19.txt', 'arq20.txt',
       'arq21.txt', 'arq22.txt', 'arq23.txt'], dtype=object)